In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 텍스트 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from konlpy.tag import Okt 
import MeCab
mecab = MeCab.Tagger()
import re 
from string import punctuation
import requests
import pickle
import ast

# 토픽모델링
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis 
import pyLDAvis.gensim

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


#### 미리 만들어둔 리뷰데이터 불러오기

In [3]:
df = pd.read_csv("D:/review.csv")

In [4]:
df

,DATE,STAR,REVIEW,LIKE
0,2019-12-27,5,부산에도 기다리던 지역화폐가 나왔네요 앱도 깔끔하고 카드도 바로 신청하고 잘 사용하...,8
1,2019-12-27,5,화이팅,4
2,2019-12-28,5,기다렸었는데 앞으로 잘 쓸게요^^,4
3,2019-12-28,5,동백전으로 여기저기 많이 사용해 볼게요. 부산 경제에도 많은 도움이 되면 좋겠습니다.,4
4,2019-12-28,5,부산 동백전 쓰기 편하게 잘 만들어졌네요 부산에서 혜택이 많았겠네요,5
5,2019-12-28,5,잘 사용해 볼게요. 동백전으로 혜택 많이 받으면 좋겠습니다.,4
6,2019-12-28,5,가자 글로벌 테스트베드 부산!,3
7,2019-12-29,5,오~ 부산도 드디어 모바일 지역화폐가 나왔네요!~ 앱설치하고 카드신청도 해봤는데요 ...,53
8,2019-12-30,5,드디어 기다리던 동백전앱이 출시됐네요 앞으로 이용 많이 하겠습니다,2
9,2019-12-30,4,군더더기 없는 ui가 마음에 드는군요. 많은 사업장에서 사용가능하면 더 좋겠습니다.,1


In [5]:
##Date column을 pandas.Datetime으로 변환
df['DATE'] = pd.to_datetime(df['DATE'])
# Date column을 index로 설정
df_new = df.set_index("DATE")

In [6]:
df_new.shape

(2871, 3)

#### corpus (말뭉치) 생성

In [7]:
## 4월을 기준으로 나누기
before_4 = df_new['2019-12-27' : '2021-04-04']
after_4 = df_new['2021-04-05': '2021-7-23']

In [8]:
corpus1 = before_4['REVIEW'] # 4월 이전 리뷰
corpus2 = after_4['REVIEW'] # 4월 이후 리뷰

#### 텍스트 전처리
- 자음모음만으로 구성된 것 제거
- 특수문자 제거
- 숫자 제거

In [9]:
def message_cleaning(docs):

  
   
    # Series의 object를 str로 변경.
    docs = [str(doc) for doc in docs]
    
    
    # 1. 자음 모음 제거하기
    pattern1 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    docs = [pattern1.sub("", doc) for doc in docs]
    # 2. 특수문자 제거
    pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
    docs = [pattern2.sub("", doc) for doc in docs]
    
    
    return docs


<>:13: DeprecationWarning: invalid escape sequence \{
<>:13: DeprecationWarning: invalid escape sequence \{
<>:13: DeprecationWarning: invalid escape sequence \{
<ipython-input-9-f6a8cd553c2c>:13: DeprecationWarning: invalid escape sequence \{
  pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")


#### 명사추출

In [10]:
import re
import MeCab # 형태소 분석기
mecab = MeCab.Tagger()
import requests
import pickle
import ast
## mecab형태소 분석기로 명사 추출하는 함수
def mecab_nouns(text): 
    nouns = []
    
    #TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) 
            for token in mecab.parse(text).splitlines()[:-1]] 
    
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        #동사(어근)까지 추출할려면 "VV"까지
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS" or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS" or token[1]== "SL" :
            nouns.append(token[0])
    return nouns

In [11]:
## 텍스트 정제
cleaned_corpus1 = message_cleaning(corpus1)
cleaned_corpus2 = message_cleaning(corpus2)

#### 불용어 처리 및 한글자 제거

In [12]:

def define_stopwords(path):
    
    SW = set()
    #불용어를 추가하는 방법 1.
    #SW.add("동백전")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word[:-1])
            
    return SW

from tqdm import tqdm_notebook # 시간 바

# 명사 추출한 것 중 SW에 포함되지 않으면서 한글자 제거
def text_tokenizing(corpus):   
    token_corpus = []
    # tqdm을 사용하여 진행 과정을 보기
    for n in tqdm_notebook(range(len(corpus))):
        token_text = mecab_nouns(corpus[n]) # 위에서 정의한 명사추출 함수 실행
        token_text = [word for word in token_text if word not in SW and len(word) >1]
        token_corpus.append(token_text)
    return token_corpus

SW = define_stopwords("D:/연구알바/stopwords-ko.txt")
SW

{'',
 'Good',
 '가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '건가',
 '건가요',
 '건지',
 '겁니까',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게요',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '경우',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '공무원',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '굿',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너무',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '

#### 사용자사전(분리되면 안되는 단어들을 따로 사전에 정의)
> 동백전
하나은행
부산은행
캐쉬백
캐시백
지역화폐
해결방안
고객센터
교통카드
체크카드
불편함
본인인증
본인 인증
인증번호
코나아이
삼성페이
오프라인
온라인
비번
비밀번호
아이디
전화번호
폰 번호
생년월일
홈페이지
소상공인
지원금
로코
큐알코드 등등

In [13]:
tokenized_text1 = text_tokenizing(cleaned_corpus1)
tokenized_text2= text_tokenizing(cleaned_corpus2)

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1856 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1015 [00:00<?, ?it/s]

#### 문서 단어 행렬

In [14]:
def dic_and_bow(clean_text):
    
    # 데이터를 dictionary 형태로 명사 list 만들기 
    dictionary = corpora.Dictionary(clean_text) 
    
    # 출현빈도가 너무 적은 단어는 제거 
    dictionary.filter_extremes(no_below=5) 
    
    # 명사 형태로 말뭉치 만들기 
    corpus = [dictionary.doc2bow(text) for text in clean_text]
    
#     TF-IDF으로 변환 
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    corpus = corpus_tfidf 
    
    return corpus ,dictionary

corpus_1 ,dictionary_1 = dic_and_bow(tokenized_text1)
corpus_2 ,dictionary_2 = dic_and_bow(tokenized_text2)

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다. 
- TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것입니다. 
- 즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 됩니다.
> 모든 문서에서 자주 등장하는 단어의 중요도를 낮게 평가 (?!)

- TF-IDF를 돌린 후 명사 추출 > corpus에서 문장에서 단어의 중요도를 평가해서 추출한 것을 다시 명사로 추출하면 그 중요도가 의미가 없어짐(중요하다는 단어가 동사일 경우 중요함에도 사라지기에)

- 명사 추출 후 TF-IDF를 돌리기 > 모든 문서에서 자주 등장하는 명사가 중요하지 않다고 판단하여 핵심 명사들이 사라지는 경우가 생김

- 단순 빈도로 인한 명사 추출뿐인가..

#### 최적 Topic 개수 산출

<span style="color:red">coherence 보단 perplexity를 우선적으로 보는 게 좋음</span>


__(1) Coherence Model__

Topic이 얼마나 의미론적으로 일관성 있는지 판단.
수치가 높을수록 일관성 높음. 0.55 정도면 우수하다고 함.
Coherence가 너무 높아지면 정보의 양이 줄어들게 되고, coherence가 너무 낮아 정보들이 인관성이 없다면 분석의 의미가 낮아지게 됨.

.3 is bad

.4 is low

.55 is okay

.65 might be as good as it is going to get

.7 is nice

.8 is unlikely and

.9 is probably wrong

- 매번 모델을 돌릴때마다 토픽이 달라지기에 seed 고정이 필요할 것 같다

__(2) 언어 모델 평가방법 (PPL: Perplexity)__

확률 모델이 결과를 얼마나 정확하게 예측하는지 나타내는 값.
동일 모델 내 파라미터에 따른 성능 평가할 때 사용.
선정된 토픽 개수마다 학습시켜 가장 낮은 값을 보이는 구간을 찾아 최적화된 토픽의 개수 선정. 
수치가 낮을수록 높은 정확도.
Coherence Score와 함께 고려해야함.

<span style="color:red">train, test set 나누기</span>

- 8:2로 나누는 게 좋다고 함. 
- 80%로 학습한 lda model을 20%의 테스트셋에 적용시켜 수치를 보고, 가장 좋은 토픽의 개수를 찾아 전체 데이터셋에 토픽모델링 적용

In [16]:
# corpus_train = corpus[:2200]
# corpus_test = corpus[2200:]

In [17]:
# perplexity_values=[]

# for i in tqdm_notebook(range(2,50)): 
#     ldamodel = gensim.models.ldamodel.LdaModel(corpus_test, 
#                                                num_topics=i, 
#                                                id2word=dictionary, 
#                                                alpha=0.1,passes = 10,iterations=1000)

#     perplexity_values.append(ldamodel.log_perplexity(corpus_test))

In [18]:
# x = range(2,100)
# plt.plot(x, perplexity_values)
# plt.xlabel("Number of Topics")
# plt.ylabel("Perplexity Score")
# plt.show()

#### 4월이전 토픽모델링 실행

In [19]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 8

NUM_TOPIC_WORDS = 10

def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# LDA를 실행.
model_1 = models.ldamodel.LdaModel(corpus_1, num_topics=NUM_TOPICS, id2word=dictionary_1, alpha=0.1,iterations=1000,random_state=100,passes=100)
model_2= models.ldamodel.LdaModel(corpus_2, num_topics=NUM_TOPICS, id2word=dictionary_2, alpha=0.1,iterations=1000,random_state=100,passes=100)



In [20]:
word_dict1 = {};

for i in range(NUM_TOPICS):

    words = model_1.show_topic(i)

    word_dict1['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict1)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,업데이트,오류,사용,가맹점,편리,연결,캐시백,혜택
1,최고,실행,만족,유용,충전,네트워크,카드,감사
2,접속,결제,10프로,사용,인증,카드,캐쉬백,포인트
3,짜증,네트워크,통신,다운,사용,데이터,지역,적립
4,강추,확인,결재,검색,가입,계좌,도움,사용
5,잔액,문제,화면,인증번호,번호,신청,할인,설치
6,자체,접속,먹통,기능,카드,와이파이,불편,기대
7,부탁,에러,적립,로그인,사람,등록,사용,불가
8,이용,서버,로딩,생각,문자,접속,금액,불편함
9,충전,안정,종료,추천,필요,발급,내역,경제


In [21]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()
# pyLDAvis 실행.
data1 = pyLDAvis.gensim.prepare(model_1, corpus_1, dictionary_1)
data1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.183413  0.043280       1        1  15.771479
4      0.000273  0.103526       2        1  15.646022
5     -0.173839  0.083522       3        1  14.426652
6      0.065912  0.101550       4        1  13.314853
0     -0.153109 -0.263063       5        1  11.267350
3      0.058448  0.052109       6        1  10.824743
7      0.214757 -0.157877       7        1   9.899910
2      0.170970  0.036954       8        1   8.848991, topic_info=      Term        Freq       Total Category  logprob  loglift
0       사용  113.000000  113.000000  Default  30.0000  30.0000
72    업데이트   53.000000   53.000000  Default  29.0000  29.0000
23      편리   42.000000   42.000000  Default  28.0000  28.0000
7       혜택   29.000000   29.000000  Default  27.0000  27.0000
138     최고   32.000000   32.000000  Default  26.0000  26.0000
68      감사   27.000000   27.000000  Default  25.0000  25.0000
44     캐시백   36.000000   36.000000  Default  24.0000  24.0000
55     가맹점   28.000000   28.000000  Default  23.0000  23.0000
64      접속   48.000000   48.000000  Default  22.0000  22.0000
80     포인트   23.000000   23.000000  Default  21.0000  21.0000
46      실행   32.000000   32.000000  Default  20.0000  20.0000
70      만족   20.000000   20.000000  Default  19.0000  19.0000
176     유용   20.000000   20.000000  Default  18.0000  18.0000
36      오류   45.000000   45.000000  Default  17.0000  17.0000
29      적립   25.000000   25.000000  Default  16.0000  16.0000
206   10프로   14.000000   14.000000  Default  15.0000  15.0000
109    캐쉬백   17.000000   17.000000  Default  14.0000  14.0000
137     연결   23.000000   23.000000  Default  13.0000  13.0000
16    네트워크   27.000000   27.000000  Default  12.0000  12.0000
24      결제   34.000000   34.000000  Default  11.0000  11.0000
10      지역   14.000000   14.000000  Default  10.0000  10.0000
6       도움   14.000000   14.000000  Default   9.0000   9.0000
139     할인   13.000000   13.000000  Default   8.0000   8.0000
111     계좌   13.000000   13.000000  Default   7.0000   7.0000
35     데이터   14.000000   14.000000  Default   6.0000   6.0000
91      인증   15.000000   15.000000  Default   5.0000   5.0000
3       카드   53.000000   53.000000  Default   4.0000   4.0000
8       기대   10.000000   10.000000  Default   3.0000   3.0000
169     짜증   10.000000   10.000000  Default   2.0000   2.0000
193     통신    9.000000    9.000000  Default   1.0000   1.0000
46      실행   30.818465   32.718000   Topic1  -2.4117   1.7872
98      에러   10.985368   11.785966   Topic1  -3.4432   1.7766
62      서버    9.204342   10.214271   Topic1  -3.6201   1.7429
208     이후    4.304576    5.105295   Topic1  -4.3801   1.6764
144     어제    4.219998    5.021253   Topic1  -4.4000   1.6731
232     환급    3.521142    4.321682   Topic1  -4.5810   1.6421
61      문제   11.435996   14.059401   Topic1  -3.4030   1.6404
197     속도    3.464324    4.264921   Topic1  -4.5973   1.6391
180     정상    3.418464    4.218954   Topic1  -4.6106   1.6366
150   마찬가지    3.028730    3.829507   Topic1  -4.7317   1.6124
78      안정    6.854840    8.671894   Topic1  -3.9149   1.6118
43      취소    3.966602    5.045123   Topic1  -4.4619   1.6065
87      오늘    3.258491    4.284317   Topic1  -4.6585   1.5733
188     알뜰    2.478614    3.279302   Topic1  -4.9321   1.5670
230    뭔가요    2.453447    3.254314   Topic1  -4.9423   1.5645
117     공지    2.382492    3.183126   Topic1  -4.9717   1.5573
229    통신사    2.353601    3.154284   Topic1  -4.9839   1.5542
120     문구    2.111241    2.911948   Topic1  -5.0925   1.5254
158    갤럭시    3.364348    4.641449   Topic1  -4.6266   1.5252
159     노트    2.848343    3.976584   Topic1  -4.7931   1.5133
36      오류   32.576350   45.804065   Topic1  -2.3562   1.5062
83      해결    6.689859    9.468377   Topic1  -3.9392   1.4996
143     동의    2.236840    3.233158   Topic1  -5.0347   1.4786
76     시스템    3.966067    5.759174   Topic1  -4.4620   1.4739
210    

#### 4월 이후 리뷰 토픽모델링 실행

In [22]:
word_dict2 = {};

for i in range(NUM_TOPICS):

    words = model_2.show_topic(i)

    word_dict2['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict2)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,이해,불편,업데이트,카드,최악,편의,금액,삼성페이
1,기능,사용,짜증,발급,예전,답변,캐시백,등록
2,인터페이스,이전,가맹점,기존,혜택,옛날,충전,결제
3,카드,기대,편리,신규,자체,문의,사용,카드
4,소득,서비스,불편함,등록,카드,사항,내역,페이
5,공제,운영,아이폰,신청,사용,사용,확인,한도
6,연동,직관,지역,쓰레기,고객,답글,불편,실물
7,진행,KT,광고,사용,최고,생각,설치,바코드
8,발급,캐시,인천,이관,먹통,kt,잔액,누가
9,ui,처음,알람,연결,적응,댓글,가맹점,기능


In [23]:
# pyLDAvis 실행.
data2 = pyLDAvis.gensim.prepare(model_2, corpus_2, dictionary_2)
data2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.194195 -0.036624       1        1  22.702503
6     -0.168003 -0.059567       2        1  22.664190
7      0.057938 -0.008517       3        1  10.983431
0      0.160277  0.094460       4        1   9.332265
2     -0.130059  0.232405       5        1   8.847596
1     -0.105678 -0.090146       6        1   8.695407
5     -0.068886 -0.086038       7        1   8.456767
4      0.060216 -0.045973       8        1   8.317841, topic_info=      Term       Freq      Total Category  logprob  loglift
94    삼성페이  28.000000  28.000000  Default  30.0000  30.0000
49      불편  54.000000  54.000000  Default  29.0000  29.0000
226   업데이트  16.000000  16.000000  Default  28.0000  28.0000
91     가맹점  24.000000  24.000000  Default  27.0000  27.0000
1       짜증  16.000000  16.000000  Default  26.0000  26.0000
3       카드  66.000000  66.000000  Default  25.0000  25.0000
65      최악  16.000000  16.000000  Default  24.0000  24.0000
0       발급  40.000000  40.000000  Default  23.0000  23.0000
67      예전  17.000000  17.000000  Default  22.0000  22.0000
145     편리  11.000000  11.000000  Default  21.0000  21.0000
53      금액  25.000000  25.000000  Default  20.0000  20.0000
71      혜택  13.000000  13.000000  Default  19.0000  19.0000
41      이전  29.000000  29.000000  Default  18.0000  18.0000
80      기대  10.000000  10.000000  Default  17.0000  17.0000
62      등록  25.000000  25.000000  Default  16.0000  16.0000
6       결제  22.000000  22.000000  Default  15.0000  15.0000
61     서비스   9.000000   9.000000  Default  14.0000  14.0000
9       기존  34.000000  34.000000  Default  13.0000  13.0000
122     내역  18.000000  18.000000  Default  12.0000  12.0000
109     기능  19.000000  19.000000  Default  11.0000  11.0000
111    불편함  15.000000  15.000000  Default  10.0000  10.0000
101    캐시백  22.000000  22.000000  Default   9.0000   9.0000
44      운영  10.000000  10.000000  Default   8.0000   8.0000
107     이해   9.000000   9.000000  Default   7.0000   7.0000
42   인터페이스   9.000000   9.000000  Default   6.0000   6.0000
21     쓰레기  15.000000  15.000000  Default   5.0000   5.0000
56      신청  18.000000  18.000000  Default   4.0000   4.0000
16      사용  55.000000  55.000000  Default   3.0000   3.0000
10      답변  10.000000  10.000000  Default   2.0000   2.0000
50      편의   8.000000   8.000000  Default   1.0000   1.0000
128   플라스틱   5.488743   6.285141   Topic1  -4.4254   1.3472
86      장난   4.184655   4.981168   Topic1  -4.6967   1.3085
55      신규  14.987847  17.916110   Topic1  -3.4209   1.3042
23      연결  10.224803  12.359191   Topic1  -3.8033   1.2931
87      환경   3.532346   4.328623   Topic1  -4.8662   1.2794
88      입력   7.185549   8.923907   Topic1  -4.1561   1.2660
64      이관  11.162906  13.970948   Topic1  -3.7155   1.2583
155    시스템   3.636677   4.571036   Topic1  -4.8371   1.2540
257     계산   3.441983   4.431776   Topic1  -4.8921   1.2299
0       발급  31.764368  40.946507   Topic1  -2.6698   1.2288
96      의무   2.731996   3.528314   Topic1  -5.1231   1.2269
251     시백   2.663228   3.459751   Topic1  -5.1486   1.2210
21     쓰레기  11.685783  15.223954   Topic1  -3.6698   1.2182
189     뭔가   2.632092   3.429585   Topic1  -5.1604   1.2180
30      정보   9.335995  12.315361   Topic1  -3.8943   1.2057
2       취소   7.108084   9.542466   Topic1  -4.1669   1.1882
163     주소   2.083451   2.883470   Topic1  -5.3941   1.1577
201     회원   2.074199   2.873045   Topic1  -5.3986   1.1569
25      유도   3.148024   4.378079   Topic1  -4.9814   1.1529
9       기존  24.626236  34.685236   Topic1  -2.9243   1.1402
56      신청  12.726252  18.076487   Topic1  -3.5845   1.1317
70      필수   3.966261   5.712403   Topic1  -4.7503   1.1179
36      해결   3.373495   4.946107   Topic1  -4.9122   1.1000
206     엉망   1.632016   2.429060   Topic1  -5.6383   1.0850
33      체크   1.846187   2.752171   Topic1  -5.5150   1.0834
3       카드  44.131620  66.140301   Topic1  -2.3410   1.07